In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import LSTM, Dropout, Dense, Activation
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
import datetime
import cx_Oracle
import os

In [24]:
# --- Oracle DB 접속 정보 (반드시 자신의 환경에 맞게 수정!) ---
DB_USERNAME = "system"  # <<--- 실제 사용자 이름으로 변경
DB_PASSWORD = "1234"    # <<--- 실제 비밀번호로 변경
DB_DSN = "localhost:1522/xe" # <<--- 실제 DSN으로 변경 (예: localhost:1521/XE 또는 localhost:1521/ORCLPDB1)

TABLE_NAME_ETH = "ETH_PRICES_DATA" # SQL Developer에서 생성한 ETH 테이블 이름 (대문자 가정)

# 모델 및 로그 저장 경로 설정
LOG_DIR = 'logs_eth' 
MODEL_SAVE_DIR = 'models_eth'

if not os.path.exists(LOG_DIR):
    os.makedirs(LOG_DIR)
    print(f"'{LOG_DIR}' 폴더 생성 완료")
if not os.path.exists(MODEL_SAVE_DIR):
    os.makedirs(MODEL_SAVE_DIR)
    print(f"'{MODEL_SAVE_DIR}' 폴더 생성 완료")

# Instant Client 경로 설정 (필요한 경우 주석 해제 및 경로 수정)
# if os.name == 'nt': # Windows 예시
#     try:
#         # cx_Oracle.init_oracle_client(lib_dir=r"C:\oracle\instantclient_19_19") # <<--- 실제 경로로 변경
#         print("Oracle Instant Client 초기화 시도 (Windows).")
#     except Exception as e:
#         print(f"Oracle Instant Client 초기화 실패 (Windows): {e}")
# elif os.name == 'posix': # macOS / Linux
#     pass # 환경 변수 (LD_LIBRARY_PATH 또는 DYLD_LIBRARY_PATH) 사용 권장

In [25]:
conn = None
data = pd.DataFrame() # 빈 DataFrame으로 초기화

try:
    conn = cx_Oracle.connect(user=DB_USERNAME, password=DB_PASSWORD, dsn=DB_DSN, encoding="UTF-8")
    print(f"Oracle DB ({DB_DSN})에 성공적으로 연결되었습니다.")
    
    # 컬럼명을 노트북의 원래 CSV 컬럼명과 유사하게 AS 처리 (따옴표로 감싸서 대소문자 구분)
    query_eth = f"""
        SELECT 
            TRADE_DATE AS "Date", 
            OPEN_PRICE AS "Open", 
            HIGH_PRICE AS "High", 
            LOW_PRICE AS "Low", 
            CLOSE_PRICE AS "Close", 
            TRADE_VOLUME_TEXT AS "Volume",
            MARKET_CAP_TEXT AS "Market Cap"
        FROM {TABLE_NAME_ETH} 
        ORDER BY TRADE_DATE ASC
    """
    data = pd.read_sql_query(query_eth, conn)
    
    if not data.empty:
        print(f"DB 테이블 '{TABLE_NAME_ETH}'에서 데이터 로드 완료.")
        
        data['Date'] = pd.to_datetime(data['Date'])
        
        def clean_eth_numeric_text(value):
            if pd.isna(value) or not isinstance(value, str):
                return float(value) if isinstance(value, (int, float)) else np.nan
            try:
                return float(value.replace(',', ''))
            except (ValueError, AttributeError):
                return np.nan
        
        data['Volume'] = data['Volume'].apply(clean_eth_numeric_text)
        data['Market Cap'] = data['Market Cap'].apply(clean_eth_numeric_text)

        for col in ['Open', 'High', 'Low', 'Close']:
             data[col] = pd.to_numeric(data[col], errors='coerce')
        
        essential_cols = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
        data.dropna(subset=essential_cols, inplace=True)
        data.reset_index(drop=True, inplace=True)

        print("\n로드 및 전처리 후 Data Head:")
        # Jupyter Notebook 환경에서는 display() 함수가 더 예쁘게 출력합니다.
        # 일반 Python 스크립트에서는 print(data.head()) 사용
        from IPython.display import display 
        display(data.head())
        print("\n로드 및 전처리 후 Data Info:")
        data.info()
    else:
        print(f"경고: DB 테이블 '{TABLE_NAME_ETH}'에서 데이터를 가져오지 못했거나 데이터가 없습니다.")

except cx_Oracle.Error as e:
    error_obj, = e.args
    print(f"Oracle DB 오류: {error_obj.code} - {error_obj.message}")
except Exception as e:
    print(f"데이터 로드 또는 처리 중 오류: {e}")
finally:
    if conn:
        conn.close()
        print("Oracle DB 연결이 닫혔습니다.")

Oracle DB (localhost:1522/xe)에 성공적으로 연결되었습니다.
DB 테이블 'ETH_PRICES_DATA'에서 데이터 로드 완료.

로드 및 전처리 후 Data Head:


C:\Users\dlgod\AppData\Local\Temp\ipykernel_4860\4102438284.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql_query(query_eth, conn)


,Date,Open,High,Low,Close,Volume,Market Cap
0,2017-10-31,307.38,310.55,305.88,305.88,369583008.0,2.933152e+10
1,2017-11-01,305.76,306.40,290.58,291.69,553864000.0,2.918359e+10
2,2017-11-02,290.73,293.91,281.17,287.43,904900992.0,2.775424e+10
3,2017-11-03,288.50,308.31,287.69,305.71,646339968.0,2.754741e+10
4,2017-11-04,305.48,305.48,295.80,300.47,416479008.0,2.917535e+10



로드 및 전처리 후 Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        365 non-null    datetime64[ns]
 1   Open        365 non-null    float64       
 2   High        365 non-null    float64       
 3   Low         365 non-null    float64       
 4   Close       365 non-null    float64       
 5   Volume      365 non-null    float64       
 6   Market Cap  365 non-null    float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 20.1 KB
Oracle DB 연결이 닫혔습니다.


In [26]:
# 변수 초기화 (이 셀이 여러 번 실행될 경우를 대비)
x_train, y_train, x_test, y_test = (None, None, None, None)
mid_prices, result, result_normalized = (None, None, None) # 중간 변수들도 초기화
seq_len = 50 

print(f"\n--- 코드 셀 4 시작 ---")
print(f"입력 data DataFrame 비어있는지: {data.empty}")
if not data.empty:
    print(f"입력 data DataFrame 행 개수: {len(data)}")

# data DataFrame이 존재하고, 필요한 'High', 'Low' 컬럼이 있는지, 그리고 데이터가 있는지 확인
if not data.empty and \
   all(col in data.columns for col in ['High', 'Low']) and \
   pd.notna(data['High']).any() and pd.notna(data['Low']).any(): # NaN만 있는 경우가 아닌지 확인

    high_prices = data['High'].values
    low_prices = data['Low'].values
    mid_prices = (high_prices + low_prices) / 2
    
    # mid_prices에 NaN이 있다면, 이후 계산에 문제가 될 수 있으므로 처리 (예: ffill 후 bfill)
    mid_prices_series = pd.Series(mid_prices)
    mid_prices_series.fillna(method='ffill', inplace=True)
    mid_prices_series.fillna(method='bfill', inplace=True)
    mid_prices_series.dropna(inplace=True) # 그래도 NaN이 있다면 해당 부분 제거
    mid_prices = mid_prices_series.values
    
    print(f"mid_prices 생성 완료 (NaN 처리 후), 길이: {len(mid_prices)}")
    
    sequence_length = seq_len + 1
    
    result = []
    # 생성 가능한 시퀀스 수 확인
    num_possible_sequences = len(mid_prices) - sequence_length + 1
    print(f"sequence_length: {sequence_length}, 생성 가능한 최대 시퀀스 수: {num_possible_sequences}")

    if num_possible_sequences > 0:
        for index in range(num_possible_sequences):
            result.append(mid_prices[index: index + sequence_length])
        print(f"생성된 시퀀스 개수 (result 리스트 길이): {len(result)}")
    else:
        print("오류: mid_prices의 길이가 sequence_length보다 짧거나 같아서 시퀀스를 생성할 수 없습니다.")
        result = [] # result를 빈 리스트로 확실히 함

    def normalize_windows(window_data):
        normalized_data_list = [] # NumPy 배열로 바로 만들기보다 리스트로 먼저 수집
        skipped_windows = 0
        for i, window_orig in enumerate(window_data):
            window = np.array(window_orig, dtype=float) # 부동소수점 연산을 위해 float으로
            if pd.isna(window[0]) or window[0] == 0:
                # print(f"경고 (윈도우 {i}): 정규화 기준값(window[0])이 0 또는 NaN. 건너뜁니다: {window[:3]}")
                skipped_windows += 1
                continue
            
            # window 내 NaN 값을 0으로 채우는 것은 정규화에 왜곡을 줄 수 있음. 
            # 여기서는 이전 단계에서 mid_prices의 NaN을 처리했다고 가정.
            # 만약 window 내에 NaN이 있다면, 해당 윈도우를 건너뛰거나 다른 방식으로 처리해야 함.
            if np.isnan(window).any():
                # print(f"경고 (윈도우 {i}): 윈도우 내에 NaN 값이 포함되어 있어 건너뜁니다: {window[:3]}")
                skipped_windows += 1
                continue

            normalized_window = [((p / window[0]) - 1) for p in window]
            normalized_data_list.append(normalized_window)
        
        print(f"normalize_windows: 입력 윈도우 {len(window_data)}개 중 건너뛴 윈도우 개수 = {skipped_windows}")
        if not normalized_data_list: # 모든 윈도우가 건너뛰어졌다면 빈 배열 반환
            return np.array([])
        return np.array(normalized_data_list)

    if result: # result 리스트에 데이터가 있는 경우
        result_normalized = normalize_windows(result)
        
        if result_normalized.size == 0 or result_normalized.shape[0] == 0: # 정규화 후 유효 데이터 없는 경우
            print("오류: 정규화 후 유효한 데이터가 없습니다. 정규화 로직 또는 입력 데이터를 확인하세요.")
        else:
            print(f"정규화된 시퀀스 형태 (result_normalized.shape): {result_normalized.shape}")
            
            # 학습/테스트 데이터 분할 (최소한 테스트셋에 1개 이상의 샘플이 있도록 조정)
            n_samples = result_normalized.shape[0]
            if n_samples < 2: # 최소 2개의 샘플은 있어야 학습/테스트 분할 의미가 있음
                print("오류: 정규화된 샘플 수가 너무 적어 학습/테스트 분할이 불가능합니다.")
            else:
                row = int(round(n_samples * 0.9))
                # 테스트셋이 최소 1개는 있도록 row 조정 (row가 전체 샘플 수와 같아지면 안 됨)
                if row >= n_samples: 
                    row = n_samples - 1 
                
                print(f"학습/테스트 분할 기준 row 인덱스 (0부터 시작): {row} (즉, train은 [:row], test는 [row:])")
            
                train = result_normalized[:row, :]
                
                if train.shape[0] > 0: # 학습 데이터가 실제로 생성되었는지 확인
                    if train.shape[0] > 1: # 1개 초과일 때만 셔플
                        np.random.shuffle(train)
                    x_train = train[:, :-1]
                    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
                    y_train = train[:, -1]
                else:
                    print("오류: train 데이터셋이 비어있습니다.")

                if n_samples > row: # 테스트 데이터셋 생성 가능 여부
                    x_test = result_normalized[row:, :-1]
                    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
                    y_test = result_normalized[row:, -1]
                else:
                    print("오류: test 데이터셋을 만들 수 없습니다 (분할할 데이터 부족).")

    else: # result 리스트가 비어있는 경우
        print("오류: 시퀀스 데이터(result)가 생성되지 않았습니다 (mid_prices 길이 부족 또는 기타 문제).")
else: # data DataFrame이 비어있거나 필요한 컬럼이 없는 경우
    print("오류: 데이터가 비어있거나 필요한 컬럼('High', 'Low')이 없어 특징 생성을 건너뜁니다.")

# 최종적으로 생성된 변수들의 상태 출력
print(f"\n--- 코드 셀 4 종료 ---")
print(f"x_train 형태: {x_train.shape if x_train is not None else 'None'}")
print(f"y_train 형태: {y_train.shape if y_train is not None else 'None'}")
print(f"x_test 형태: {x_test.shape if x_test is not None else 'None'}")
print(f"y_test 형태: {y_test.shape if y_test is not None else 'None'}")

if x_train is not None and np.isnan(x_train).any(): print("경고: x_train에 NaN 값이 포함되어 있습니다!")
if y_train is not None and np.isnan(y_train).any(): print("경고: y_train에 NaN 값이 포함되어 있습니다!")
if x_test is not None and np.isnan(x_test).any(): print("경고: x_test에 NaN 값이 포함되어 있습니다!")
if y_test is not None and np.isnan(y_test).any(): print("경고: y_test에 NaN 값이 포함되어 있습니다!")


--- 코드 셀 4 시작 ---
입력 data DataFrame 비어있는지: False
입력 data DataFrame 행 개수: 365
mid_prices 생성 완료 (NaN 처리 후), 길이: 365
sequence_length: 51, 생성 가능한 최대 시퀀스 수: 315
생성된 시퀀스 개수 (result 리스트 길이): 315
normalize_windows: 입력 윈도우 315개 중 건너뛴 윈도우 개수 = 0
정규화된 시퀀스 형태 (result_normalized.shape): (315, 51)
학습/테스트 분할 기준 row 인덱스 (0부터 시작): 284 (즉, train은 [:row], test는 [row:])

--- 코드 셀 4 종료 ---
x_train 형태: (284, 50, 1)
y_train 형태: (284,)
x_test 형태: (31, 50, 1)
y_test 형태: (31,)


C:\Users\dlgod\AppData\Local\Temp\ipykernel_4860\2116145814.py:22: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  mid_prices_series.fillna(method='ffill', inplace=True)
C:\Users\dlgod\AppData\Local\Temp\ipykernel_4860\2116145814.py:23: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  mid_prices_series.fillna(method='bfill', inplace=True)


In [27]:
model = None 
if x_train is not None and x_train.shape[0] > 0 and y_train is not None: 
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(seq_len, 1)))
    # model.add(Dropout(0.2))
    model.add(LSTM(64, return_sequences=False))
    # model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer='rmsprop') # 원본 노트북에서는 rmsprop 사용
    print("\n모델 요약:")
    model.summary()
else:
    print("학습 데이터가 충분하지 않아 모델을 생성할 수 없습니다.")


모델 요약:
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 50, 50)            10400     
                                                                 
 lstm_7 (LSTM)               (None, 64)                29440     
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 39905 (155.88 KB)
Trainable params: 39905 (155.88 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history = None # history 변수를 셀 시작 시 None으로 초기화
if model is not None and \
   x_train is not None and x_train.shape[0] > 0 and \
   y_train is not None and y_train.shape[0] > 0 and \
   x_test is not None and x_test.shape[0] > 0 and \
   y_test is not None and y_test.shape[0] > 0: # 모든 필요한 데이터가 준비되었는지 확인

    start_time = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')

    # 콜백 정의
    tensorboard_callback = TensorBoard(log_dir=os.path.join(LOG_DIR, start_time))
    
    # 모델 파일명에 심볼 이름 포함 (예: eth)
    model_filename = f'{start_time}_eth_best_model.h5' # 이 부분은 crypto_eth.ipynb 용입니다.
                                                      # stock_samsung.ipynb 에서는 _samsung_ 등으로 변경
    checkpoint_callback = ModelCheckpoint(
        filepath=os.path.join(MODEL_SAVE_DIR, model_filename), 
        monitor='val_loss', 
        verbose=1, 
        save_best_only=True, 
        mode='min'
    )
    reduce_lr_callback = ReduceLROnPlateau(
        monitor='val_loss', 
        factor=0.2, 
        patience=5, 
        verbose=1, 
        mode='min',
        min_lr=1e-7
    )

    print("\n모델 학습 시작...")
    history = model.fit( # <<--- 바로 이 라인에서 history 변수에 학습 결과가 할당됩니다!
        x_train, 
        y_train,
        validation_data=(x_test, y_test),
        batch_size=10, 
        epochs=20,     
        callbacks=[tensorboard_callback, checkpoint_callback, reduce_lr_callback]
    )
    print("모델 학습 완료.")
else:
    print("모델 또는 학습/검증 데이터가 충분하지 않아 학습을 진행할 수 없습니다.")
    # 이 경우 history는 위에서 None으로 초기화된 상태를 유지합니다.
    # 디버깅을 위해 각 변수의 상태를 출력해볼 수 있습니다.
    print(f"model is None: {model is None}")
    print(f"x_train is None or empty: {x_train is None or (x_train is not None and x_train.shape[0] == 0)}")
    print(f"y_train is None or empty: {y_train is None or (y_train is not None and y_train.shape[0] == 0)}")
    print(f"x_test is None or empty: {x_test is None or (x_test is not None and x_test.shape[0] == 0)}")
    print(f"y_test is None or empty: {y_test is None or (y_test is not None and y_test.shape[0] == 0)}")

모델, 테스트 데이터, 또는 학습 이력이 없어 예측 및 시각화를 수행할 수 없습니다.
